In [1]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph()

In [2]:
learning_rate =0.1

x_data =[[0,0],
         [0,1],
         [1,0],
         [1,1]]


y_data = [[0],[1],[1],[0]]

x_data = np.array(x_data, dtype=np.float32)
y_data = np.array(y_data, dtype=np.float32)

In [3]:
with tf.name_scope ("input") as scope :
    X= tf.placeholder(tf.float32, [None,2])

with tf.name_scope ("Y") as scope :    
    Y= tf.placeholder(tf.float32, [None,1])

In [4]:
with tf.name_scope("layer1") as scope:
    W1= tf.Variable(tf.random_normal([2,10]),name='weight1')
    b1 = tf.Variable(tf.random_normal([10]),name='bias1')
    L1 =tf.sigmoid(tf.matmul(X,W1)+b1)

    w1_hist = tf.summary.histogram("weight1",W1)
    b1_hist = tf.summary.histogram("bias1",b1)
    L1_hist = tf.summary.histogram("layer1",L1)

with tf.name_scope("layer2") as scope:    
    W2= tf.Variable(tf.random_normal([10,10]),name='weight2')
    b2 = tf.Variable(tf.random_normal([10]),name='bias2')
    L2 =tf.sigmoid(tf.matmul(L1,W2)+b2)

    w2_hist = tf.summary.histogram("weight2",W2)
    b2_hist = tf.summary.histogram("bias2",b2)
    L2_hist = tf.summary.histogram("layer2",L2)
    
with tf.name_scope("layer3") as scope:    
    W3= tf.Variable(tf.random_normal([10,10]),name='weight3')
    b3 = tf.Variable(tf.random_normal([10]),name='bias3')
    L3 =tf.sigmoid(tf.matmul(L2,W3)+b3)

    w3_hist = tf.summary.histogram("weight3",W3)
    b3_hist = tf.summary.histogram("bias3",b3)
    L3_hist = tf.summary.histogram("layer3",L3)

with tf.name_scope("hypothesis") as scope:    
    W4= tf.Variable(tf.random_normal([10,1]),name='weight4')
    b4 = tf.Variable(tf.random_normal([1]),name='bias4')
    hypothesis =tf.sigmoid(tf.matmul(L3,W3)+b4)
    
    w4_hist = tf.summary.histogram("weight3",W4)
    b4_hist = tf.summary.histogram("bias3",b4)
    hypothesis_hist = tf.summary.histogram("hypothesis",hypothesis)

In [5]:
with tf.name_scope("cost") as scope:
    cost = - tf.reduce_mean(Y*tf.log(hypothesis)+(1-Y)*tf.log(hypothesis))
    ost_summ = tf.summary.scalar("cost",cost)

In [6]:
with tf.name_scope("train") as scope:
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [7]:
predicted = tf.cast(hypothesis>0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted,Y),dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy",accuracy)

In [9]:
with tf.Session() as sess:
    merged_summmary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_logs")
    writer.add_graph(sess.graph)
    
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        summary, _ =  sess.run([merged_summmary,train], feed_dict={X: x_data, Y: y_data})
        writer.add_summary(summary,global_step=step)
        if step %1000 ==0:
            print(step, sess.run(cost,feed_dict={X: x_data, Y: y_data}), sess.run([W1, W2]))
            
    h,c,a = sess.run([hypothesis, predicted,accuracy], feed_dict={X: x_data, Y: y_data})
    print("\n Hypothesis:",h,"\ncorrect:",c,"\nAccuaracy:",a)

0 1.0245 [array([[-0.44469362,  0.17703722,  0.41078764,  0.90241617,  1.49079454,
        -0.11386325, -1.62957025, -0.8316772 ,  0.42226094, -0.02314857],
       [-1.3043077 , -1.65807092, -1.66460371, -0.83231437,  0.04254093,
         0.99207687,  1.04877055, -0.02175159, -1.30451727,  0.56827438]], dtype=float32), array([[ -4.13419724e-01,   1.74458230e+00,   1.29392505e+00,
         -7.50700057e-01,  -1.31488907e+00,   1.80664289e+00,
          1.34256303e-01,  -2.53972411e+00,   4.86137688e-01,
         -2.17711180e-03],
       [ -6.51641935e-03,  -1.90231621e+00,   6.42623782e-01,
         -3.05680871e-01,   8.10960412e-01,  -9.96227935e-02,
         -5.95015645e-01,  -4.75932062e-01,   3.64963472e-01,
         -1.40824425e+00],
       [ -1.84415329e+00,  -6.43602610e-01,  -1.18434680e+00,
         -1.44378781e-01,  -8.00476253e-01,   2.85555720e-01,
         -6.12929501e-02,  -3.72809350e-01,   1.62033951e+00,
          2.06466585e-01],
       [ -1.19797909e+00,   1.43115252e-

4000 1.33813e-06 [array([[ 0.32452521,  1.49469721, -0.62993598,  0.21977653,  1.49427032,
        -0.94422048, -2.5985837 , -1.01363659, -0.52772546,  0.90184438],
       [-0.56403393, -0.01190107, -2.68863034, -1.55952895, -0.28326517,
         0.17767085,  0.03963756, -0.08827091, -1.91602528,  1.3998487 ]], dtype=float32), array([[  6.16185606e-01,   1.53916037e+00,   2.32205963e+00,
          4.34577353e-02,   3.47297370e-01,   2.92250228e+00,
          1.16655886e+00,  -3.72487783e+00,   1.13931894e+00,
         -5.50071061e-01],
       [  1.01432359e+00,  -2.27205610e+00,   1.65970492e+00,
          4.53646183e-01,   2.44787478e+00,   1.01216304e+00,
          4.85291123e-01,  -1.58941865e+00,   1.13345015e+00,
         -1.98091447e+00],
       [ -8.31887245e-01,  -1.22806802e-01,  -3.59798461e-01,
          5.88132679e-01,   6.06998026e-01,   1.35718822e+00,
          9.51930881e-01,  -1.27404070e+00,   2.15057635e+00,
         -4.26259398e-01],
       [ -1.80089384e-01,   4.07

8000 1.78814e-07 [array([[ 0.30425018,  1.53130984, -0.62743169,  0.18404953,  1.58269346,
        -0.93679899, -2.59910583, -1.02135456, -0.57386428,  0.91768879],
       [-0.58407438,  0.08166766, -2.687181  , -1.58411276, -0.17847361,
         0.19116195,  0.03726295, -0.09761952, -1.9379921 ,  1.41059744]], dtype=float32), array([[  6.08910978e-01,   1.36102235e+00,   2.32301331e+00,
          2.86082714e-03,   3.40946734e-01,   2.92925477e+00,
          1.19837165e+00,  -3.72721171e+00,   1.17179811e+00,
         -5.86435020e-01],
       [  1.00403345e+00,  -2.51915121e+00,   1.66113544e+00,
          3.84904921e-01,   2.43817854e+00,   1.02350044e+00,
          5.31007349e-01,  -1.59282410e+00,   1.18265676e+00,
         -2.03187561e+00],
       [ -8.32993090e-01,  -1.68652818e-01,  -3.59668076e-01,
          5.79713702e-01,   6.05918467e-01,   1.35814190e+00,
          9.60528970e-01,  -1.27451754e+00,   2.15437365e+00,
         -4.35183495e-01],
       [ -1.83457360e-01,   3.12